In [9]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

query = """
SELECT 
    checker.uid,
    checker.labname,
    checker.timestamp AS first_commit_ts,
    MIN(pageviews.datetime) AS first_view_ts
FROM checker
LEFT JOIN pageviews ON checker.uid = pageviews.uid
WHERE 
    checker.status = 'ready'
    AND checker.numTrials = 1
    AND checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY checker.uid, checker.labname
"""

datamart = pd.read_sql_query(query, conn, parse_dates=['first_commit_ts', 'first_view_ts'])

test = datamart[datamart['first_view_ts'].notna()].copy()
control = datamart[datamart['first_view_ts'].isna()].copy()

avg_view_ts = test['first_view_ts'].mean()
control.loc[:, 'first_view_ts'] = control['first_view_ts'].fillna(avg_view_ts)

test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

conn.close()

In [4]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [6]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [7]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
test = pd.io.sql.read_sql('SELECT * FROM test', conn)
conn.close()

In [8]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
control = pd.io.sql.read_sql('SELECT * FROM control', conn)
conn.close()